In [1]:
%pylab inline

from tqdm import tqdm_notebook as tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2021
DAY = int('16')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

Populating the interactive namespace from numpy and matplotlib


In [19]:
lines, groups = linesplit("""
EE00D40C823060
38006F45291200
D2FE28
8A004A801A8002F478
620080001611562C8802118E34
C0015000016115A2E0802F182340
A0016C880162017C3686B18A3D4780
""".strip())

In [35]:
lines, groups = linesplit(raw_input, verbose=True)

1 lines
1 groups
>>>
60552F100693298A9EF0039D24B129BA56D67282E600A4B5857002439CE580E5E5AEF67803600D2E294B2FCE8AC489BAEF37...


In [31]:
def parsehex(s):
  for i in s:
    value = int(i, 16)
    yield value // 8 % 2
    yield value // 4 % 2
    yield value // 2 % 2
    yield value // 1 % 2
    
def bit2int(a):
  n = len(a)
  return np.dot(a, (2 ** arange(n))[::-1])

def parse(s):
  print('parsing', s)
  return parsebits(list(parsehex(s)))

versions = []
  
def parsebits(a):
  global versions
  print('parsing', a)
  v = bit2int(a[0:3])
  versions.append(v)
  t = bit2int(a[3:6])
  i = 6
  
  if t == 4:
    # literal value
    lit = []
    while True:
      if a[i] == 1:
        lit += a[i + 1 : i + 5]
        i += 5
      if a[i] == 0:
        lit += a[i + 1 : i + 5]
        i += 5
        break
    x = bit2int(lit)
    print(i, 'lit', x)
    return i, 'lit', x
  # operator
  else:
    subs = []
    if a[6] == 0: # bitlength
      bitlength = bit2int(a[7:7+15])
      b = a[7+15:7+15+bitlength]
      
      bi = 0
      while bi < len(b):
        bi, sub_type, sub_x = parsebits(b)
        subs.append((sub_type, sub_x))
        b = b[bi:]
        bi = 0

      return 7+15+bitlength, 'op', subs
    elif a[6] == 1: # packetlength
      packetlength = bit2int(a[7:7+11])
      i = 7 + 11
      
      for packet_i in range(packetlength):
        print(packet_i, packetlength)
        bi, sub_type, sub_x = parsebits(a[i:])
        i += bi
        subs.append((sub_type, sub_x))
      
      return i, 'op', subs
      
  print(v)
  
  
# list(parse(lines[0]))

In [40]:
#for line in lines:
#  parse(line)
#   print('!!!', sum(versions))
#   versions = []

In [38]:
submit1(877)

True

In [53]:
def parsehex(s):
  for i in s:
    value = int(i, 16)
    yield value // 8 % 2
    yield value // 4 % 2
    yield value // 2 % 2
    yield value // 1 % 2
    
def bit2int(a):
  n = len(a)
  return np.dot(a, (2 ** arange(n))[::-1])

def parse(s):
#   print('parsing', s)
  return parsebits(list(parsehex(s)))

versions = []

def op(t, args):
  if t == 0:
    return sum(args)
  elif t == 1:
    return product(args)
  elif t == 2:
    return min(args)
  elif t == 3:
    return max(args)
  elif t == 5:
    a, b = args
    return 1 if a > b else 0
  elif t == 6:
    a, b = args
    return 1 if a < b else 0
  elif t == 7:
    a, b = args
    return 1 if a == b else 0
  else:
    raise Exception()
  
def parsebits(a):
  global versions
#   print('parsing', a)
  v = bit2int(a[0:3])
  versions.append(v)
  t = bit2int(a[3:6])
  i = 6
  
  if t == 4:
    # literal value
    lit = []
    while True:
      if a[i] == 1:
        lit += a[i + 1 : i + 5]
        i += 5
      if a[i] == 0:
        lit += a[i + 1 : i + 5]
        i += 5
        break
    x = bit2int(lit)
    print(i, 'lit', x)
    return i, 'lit', x
  # operator
  else:
    subs = []
    if a[6] == 0: # bitlength
      bitlength = bit2int(a[7:7+15])
      b = a[7+15:7+15+bitlength]
      
      bi = 0
      while bi < len(b):
        bi, sub_type, sub_x = parsebits(b)
        subs.append((sub_type, sub_x))
        b = b[bi:]
        bi = 0
#       print(subs)
      return 7+15+bitlength, 'op', op(t, [sub[1] for sub in subs])
    elif a[6] == 1: # packetlength
      packetlength = bit2int(a[7:7+11])
      i = 7 + 11
      
      for packet_i in range(packetlength):
        print(packet_i, packetlength)
        bi, sub_type, sub_x = parsebits(a[i:])
        i += bi
        subs.append((sub_type, sub_x))
      
#       print(subs)
      return i, 'op', op(t, [sub[1] for sub in subs])
      
  print(v)
  
  
list(parse(lines[0]))

26 lit 18519
16 lit 43
41 lit 78927470
11 lit 2
0 2
21 lit 1634
1 2
26 lit 52320
46 lit 1923994496
11 lit 10
16 lit 107
16 lit 210
11 lit 9
51 lit 48781326955
11 lit 13
16 lit 226
31 lit 315663
36 lit 15396305
16 lit 100
31 lit 474569
0 3
16 lit 68
1 3
11 lit 15
2 3
31 lit 404636
0 2
36 lit 2819168
41 lit 32479852
1 2
21 lit 1953
0 3
16 lit 191
1 3
16 lit 55
2 3
16 lit 212
0 1
21 lit 1037
21 lit 2156
0 1
21 lit 1505
46 lit 2381514313
0 2
0 2
31 lit 344095
1 2
31 lit 234578
1 2
21 lit 3383
11 lit 11
0 3
11 lit 7
1 3
11 lit 3
2 3
11 lit 11
0 3
11 lit 13
1 3
11 lit 11
2 3
11 lit 10
11 lit 3
11 lit 7
11 lit 7
0 2
21 lit 765
1 2
11 lit 7
61 lit 4399446759194
21 lit 946
0 1
16 lit 252
0 2
26 lit 37028
1 2
26 lit 37028
11 lit 1
0 2
0 2
11 lit 11
11 lit 6
11 lit 4
1 2
11 lit 15
11 lit 2
11 lit 11
1 2
31 lit 378529
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
51 lit 53272880723
31 lit 206576
21 lit 2963
16 lit 115
0 2
16 lit 250
1 2
21 lit 1119
16 lit 142
0 2
31 lit 113083
1 2
36 lit 9388306
26 lit 5515

[5473, 'op', 194435634456]

In [54]:
submit2(194435634456)

True